In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [2]:
hand_landmarks_dataset = pd.read_csv('E:\Sharvil\Code\PBL SEM IV\Computer Vision\\final_hand_landmarks.csv')

In [3]:
landmarks_train = hand_landmarks_dataset.iloc[:,:-1]
labels_train = hand_landmarks_dataset[['class']]
class_names = ['noGesture','ThumbsDown','ThumbsUp']

In [4]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(63,)),
    keras.layers.Dense(45,activation='relu'),
    keras.layers.Dense(45,activation='relu'),
    keras.layers.Dense(45,activation='relu'),
    keras.layers.Dense(3,activation='softmax')])

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
model.fit(landmarks_train,labels_train,epochs=15)

Epoch 1/15


625/625 [==============================] - 2s 2ms/step - loss: 0.4399 - accuracy: 0.7979
Epoch 2/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1146 - accuracy: 0.9650
Epoch 3/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0756 - accuracy: 0.9759
Epoch 4/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0533 - accuracy: 0.9839
Epoch 5/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0451 - accuracy: 0.9861
Epoch 6/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0323 - accuracy: 0.9908
Epoch 7/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0315 - accuracy: 0.9907
Epoch 8/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0307 - accuracy: 0.9908
Epoch 9/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0245 - accuracy: 0.9924
Epoch 10/15
625/625 [==============================] - 1s 2ms/step - loss: 0.0206 - accuracy: 0.99

In [1]:
import joblib

In [2]:
joblib.dump(model, 'E:\Sharvil\Code\PBL SEM IV\Computer Vision\ml_model.pkl')

NameError: name 'model' is not defined

In [3]:
loaded_model = joblib.load('E:\Sharvil\Code\PBL SEM IV\Computer Vision\ml_model.pkl')
class_names = ['noGesture','ThumbsDown','ThumbsUp']

In [4]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import pyautogui
import win32gui
import win32process
import win32api
import win32con
import os
import time

In [6]:
def get_active_browser_title():
    window = win32gui.GetForegroundWindow()
    pid = win32process.GetWindowThreadProcessId(window)[1]
    handle = win32api.OpenProcess(win32con.PROCESS_QUERY_INFORMATION | win32con.PROCESS_VM_READ, False, pid)
    exe_path = win32process.GetModuleFileNameEx(handle, 0)
    return os.path.basename(exe_path)

def is_doc_viewer_application(application):
    doc_viewer_application_list = ['explorer.exe', 'vlc.exe', 'AcroRd32.exe', 'FoxitReader.exe', 'NotroPDFReader.exe', 'SumatraPDF.exe', 'PDFXEdit.exe', 'WINWORD.EXE', 'EXCEL.EXE', 'POWERPNT.EXE', 'Acrobat.exe', 'notepad.exe', 'write.exe']
    return any(app.lower() in application.lower() for app in doc_viewer_application_list)




# MediaPipe initialization
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Video capture initialization
cam = cv.VideoCapture(0)

thumbs_up = False
thumbs_up_2 = False
thumbs_up_3 = 0

thumbs_down = False
thumbs_down_2 = False
thumbs_down_3=0
tic_up = 0
tic_down = 0

org = (50, 50)  # Origin point for text
font = cv.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 255, 255)  # White color
thickness = 2
lineType = cv.LINE_AA

def thumbsUp():
    global thumbs_up, thumbs_up_2, thumbs_up_3, tic_up
    if thumbs_up:
        if time.perf_counter() - tic_up > 1:
            thumbs_up_2 = True
            thumbs_up_3+=1
        
    else:
        thumbs_up = True
        tic_up = time.perf_counter()

    if thumbs_up_2:
        if thumbs_up_3 == 1:
            if active_browser_title.lower() == "vlc.exe":
                pyautogui.press("space")

            else:
                pyautogui.press("right")
                                
            thumbs_up = False
            thumbs_up_2 = False
            thumbs_up_3 = 0
            tic_up = 0
        
                    
    # Reset thumbs-down variables when thumbs-up is detected
    thumbs_down = False
    thumbs_down_2 = False
    thumbs_down_3=0


def thumbsDown():
    global thumbs_down, thumbs_down_2, thumbs_down_3, tic_down
    if thumbs_down:
        if time.perf_counter() - tic_down > 1:
            thumbs_down_2 = True
            thumbs_down_3+=1
        
    else:
        thumbs_down = True
        tic_down = time.perf_counter()

    if thumbs_down_2:
        if thumbs_down_3 == 1:    
            pyautogui.press("left")
            
            thumbs_down = False
            thumbs_down_2 = False
            thumbs_down_3=0
            tic_down = 0
            
                    
    # Reset thumbs-up variables when thumbs-down is detected
    thumbs_up = False
    thumbs_up_2 = False
    thumbs_up_3=0


def ignore():
    global thumbs_up, thumbs_up_2, thumbs_up_3, thumbs_down, thumbs_down_2, thumbs_down_3
    
    thumbs_up=False
    thumbs_up_2 = False
    thumbs_up_3=0 
    thumbs_down = False
    thumbs_down_2 = False
    thumbs_down_3=0    

# Main loop for capturing hand landmarks and making predictions
while True:
    success, frame = cam.read()
    if not success:
        print("No frame to show")
        continue
    
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    hands_detected = hands.process(frame_rgb)
    frame = cv.cvtColor(frame_rgb, cv.COLOR_RGB2BGR)
    
    if hands_detected.multi_hand_landmarks:
        for hand_landmarks in hands_detected.multi_hand_landmarks:
            # Extract landmarks
            landmark_row = []
            for landmark in hand_landmarks.landmark:
                landmark_row.extend([landmark.x, landmark.y, landmark.z])
            
            # Fill empty cells if less than 21 landmarks detected
            while len(landmark_row) < 63:
                landmark_row.extend([None, None, None])
            
            # Convert to numpy array
            hand_landmarks = np.array([landmark_row])


            
                
                # Predict using the trained model
            predictions = model.predict(hand_landmarks)
            predicted_class_index = np.argmax(predictions)
            predicted_gesture = class_names[predicted_class_index]
                
                # Print predictions on the screen
                
            
                #frame = cv.putText(frame, predicted_gesture, org, font, fontScale, color, thickness, lineType)
                #print(predicted_gesture)
                
                # Define a dictionary mapping gesture indices to actions
                '''
                gesture_actions = {
                    0: lambda: ignore(),  # No action for case 0
                    1: lambda: thumbsDown(),
                    2: lambda: thumbsUp()  # Assuming pattern-3 corresponds to index 2
                }
                '''
                

            frame = cv.putText(frame, predicted_gesture, org, font, fontScale, color, thickness, lineType)

            
    
    cv.imshow("Captures", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cam.release()
cv.destroyAllWindows()

NameError: name 'predicted_gesture' is not defined